<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/Eye_ai_Image_Cropping_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Prerequisites
!pip install --upgrade --force pydantic git+https://github.com/fair-research/bdbag git+https://github.com/informatics-isi-edu/deriva-py git+https://github.com/informatics-isi-edu/eye-ai-tools git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
import json
import os
from eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging


In [ ]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = True #@param ["False", "True"] {type:"raw"}
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


In [ ]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id, data_dir="test/")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)


In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid="2-5QF8")
configuration_records.model_dump()

In [ ]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
field2_csv_path = EA.filter_angle_2(configuration_records.bag_paths[0])
field2_csv_path

In [ ]:
# @title Execute Proecss algorithm (Cropping)
from eye_ai_ml.glaucoma.optic_disk_crop import preprocess_and_crop
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  preprocess_and_crop(
      str(configuration_records.bag_paths[0])+"/data/assets/Image/",
      field2_csv_path,
      str(EA.data_dir)+'/output.csv',
      'template.jpg',
      str(EA.data_dir),
      configuration_records.assets_paths[0],
      configuration_records.vocabs['Annotation_Type'][0].rid,
      configuration_records.vocabs['Annotation_Type'][0].name,
      False
      )

In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid)

In [ ]:
# @title Save cropping metadata to Image Annotation table
annot_metadata = pd.read_csv(str(EA.data_dir)+'/output.csv')
EA.insert_image_annotation(uploaded_assets[str(EA.data_dir)+'/Execution_Assets/Image_Annotation'], annot_metadata)